<a href="https://colab.research.google.com/github/stellagerantoni/learning-time-series-counterfactuals/blob/main/multivariate_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 ! git clone https://github.com/stellagerantoni/learning-time-series-counterfactuals
 %cd learning-time-series-counterfactuals/

Cloning into 'learning-time-series-counterfactuals'...
remote: Enumerating objects: 306, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 306 (delta 119), reused 24 (delta 24), pack-reused 138
Receiving objects: 100% (306/306), 4.36 MiB | 11.76 MiB/s, done.
Resolving deltas: 100% (188/188), done.
/content/learning-time-series-counterfactuals


In [2]:

!pip install -q wildboar
!pip install -q scikit-learn
!pip install -q stumpy
!pip install -q fastdtw
!pip install aeon[all_extras]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.1/169.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
import logging
import os
import warnings
from argparse import ArgumentParser
from aeon.datasets import load_classification

from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.spatial import distance_matrix
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree, KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from wildboar.datasets import load_dataset
from wildboar.ensemble import ShapeletForestClassifier
from wildboar.explain.counterfactual import counterfactuals

from _composite import ModifiedLatentCF
%cd src
from _vanilla import LatentCF
from help_functions import (ResultWriter, conditional_pad, evaluate,
                            find_best_lr, plot_graphs,
                            reset_seeds, time_series_normalize,
                            time_series_revert, upsample_minority,
                            validity_score)
from keras_models import *

/content/learning-time-series-counterfactuals/src


In [4]:
os.environ['TF_DETERMINISTIC_OPS'] = '1'
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [24]:

X, y, meta_data = load_classification("FaceDetection")
print(" Shape of X = ", X.shape)
print(" Meta data = ", meta_data)
print(f"\nshape of X is {X.shape}\n")
print(X[:3])

 Shape of X =  (9414, 144, 62)
 Meta data =  {'problemname': 'facedetection', 'timestamps': False, 'missing': False, 'univariate': False, 'equallength': True, 'classlabel': True, 'targetlabel': False, 'class_values': ['0', '1']}

shape of X is (9414, 144, 62)

[[[-0.07545  -0.336703 -0.278238 ... -0.411078 -1.016122 -1.161735]
  [ 0.05608  -0.128013 -0.323847 ... -2.114348  0.208789 -0.509533]
  [-0.824537 -0.746068 -0.482871 ... -0.929275 -1.007972 -0.292018]
  ...
  [-0.56758  -1.073942 -1.136367 ... -0.541122 -0.765445 -1.73308 ]
  [-0.23404   0.104291  0.327425 ... -1.458801  0.318952  1.854007]
  [-0.356189 -0.511199 -0.483072 ... -1.177502 -0.728301 -0.400074]]

 [[-0.093192 -0.310616 -0.388905 ...  1.937879  0.978442  0.626839]
  [-1.584982 -1.138981 -0.852494 ... -0.334188  0.636669  0.078561]
  [-0.462859 -0.301394 -0.50619  ...  0.864706  0.113177 -0.337559]
  ...
  [-0.056147 -0.051994 -0.195028 ... -2.212319 -1.052363 -1.774344]
  [-0.71979  -0.624242 -0.548486 ...  0.27643

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)